This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, Simulator, StartMeshcat

from manipulation import ConfigureParser, running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# PR2 model example

First we'll use the ModelVisualizer to inspect the model (note that [the sliders in ModelVisualizer do not currently respect the kinematic "mimic" joint in the fingers](https://github.com/RobotLocomotion/drake/issues/18917))

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.AddModels(
    url="package://drake_models/pr2_description/urdf/pr2_simplified.urdf"
)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

Now we can use HardwareStation to create a basic simulation.

In [ ]:
scenario_data = """
directives:
- add_model:
    name: pr2
    file: package://drake_models/pr2_description/urdf/pr2_simplified.urdf
model_drivers:
    pr2: !JointStiffnessDriver
        gains:
            # TODO(russt): tune these!
            x_motor:
                kp: 600
                kd: 120
            y_motor:
                kp: 600
                kd: 120
            theta_motor:
                kp: 600
                kd: 120
            torso_lift_motor:
                kp: 600
                kd: 120
            head_pan_motor:
                kp: 100
                kd: 20
            head_tilt_motor:
                kp: 100
                kd: 20
            r_upper_arm_roll_motor:
                kp: 600
                kd: 120
            r_shoulder_pan_motor:
                kp: 600
                kd: 120
            r_shoulder_lift_motor:
                kp: 600
                kd: 120
            r_forearm_roll_motor:
                kp: 400
                kd: 80
            r_elbow_flex_motor:
                kp: 400
                kd: 80
            r_wrist_flex_motor:
                kp: 200
                kd: 40
            r_wrist_roll_motor:
                kp: 200
                kd: 40
            r_gripper_l_finger_motor:
                kp: 100
                kd: 20
            l_upper_arm_roll_motor:
                kp: 600
                kd: 120
            l_shoulder_pan_motor:
                kp: 600
                kd: 120
            l_shoulder_lift_motor:
                kp: 600
                kd: 120
            l_forearm_roll_motor:
                kp: 400
                kd: 80
            l_elbow_flex_motor:
                kp: 400
                kd: 80
            l_wrist_flex_motor:
                kp: 200
                kd: 40
            l_wrist_roll_motor:
                kp: 200
                kd: 40
            l_gripper_l_finger_motor:
                kp: 100
                kd: 20
"""

scenario = LoadScenario(data=scenario_data)
station = MakeHardwareStation(scenario, meshcat)
simulator = Simulator(station)
context = simulator.get_mutable_context()

plant = station.GetSubsystemByName("plant")
pr2 = plant.GetModelInstanceByName("pr2")

mimic_joints = [
    "gripper_r_finger_joint",
    "gripper_l_finger_tip_joint",
    "gripper_r_finger_tip_joint",
    "gripper_r_finger_joint",
    "gripper_l_finger_tip_joint",
    "gripper_r_finger_tip_joint",
]
x0 = station.GetOutputPort("pr2.state_estimated").Eval(context)
x0_wout_mimics = []
for i, state_name in enumerate(plant.GetStateNames(pr2)):
    if all(mimic not in state_name for mimic in mimic_joints):
        x0_wout_mimics.append(x0[i])

station.GetInputPort("pr2.desired_state").FixValue(context, x0_wout_mimics)
simulator.AdvanceTo(0.1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>